### lec 5-1 : logistic classification 의 가설 함수 정의

분류의 종류
- 스팸 메일
- 페이스북 피드
- 신용카드 사기 거래

1 또는 0 으로 예측. 

주식 시장의 상승 하락을 분류 할 수 도 있음.

$$ z = Wx + b$$
를 만들어 g(z) 가 0에서 1사이의 결과값을 내주는 함수를 찾음  
#### sigmoid(or logistic function)  
$$g(x) = \frac{1}{(1+e^{-W^TX})}$$

항상 0과 1사이의 값을 가질 수 있게 됨.  
logistic 함수의 가장 중요한 부분

![image.png](img/5-1.png)
- 함수가 linear regression일 때는 cost function 의 그래프가 2차 포물선으로 비교적 단순
- But, logistic 함수의 경우에는 cost function 의 그래프가 오른쪽과 같이 울퉁불퉁
- logistic cost 함수의 경우 local minimum에 걸려, global minimum을 못찾을 수 있음.
- 따라서, hypothesis를 바꿨으므로 cost function도 바꿔야함

#### new cost function for logistic
$$  cost(W) = \frac{1}{m}\sum c(H(x), y)$$
$$  c(H(x), y) = 
\begin{cases}
-log(H(x)), &\mbox{ : y = 1} \\
-log(1 - H(x)), &\mbox{ : y = 0}
\end{cases}$$

c함수는 y 가 1, 0일 경우로 2가지로 나눠서 정의.  
exp로 구부러진 함수를 log함수로 잡는 것이 기본적인 아이디어

$$C(H(x), y) = ylog(H(x)) - (1-y)log(1-H(x))$$

#### Minimize cost - Gradient descent algorithm
가중치에서 alpha를 곱해준 cost(W)의 미분값을 빼줌  
(텐서플로우의 경우 gradientdescentoptimizer 를 사용하면 자동 해결)  

### lec 6-1 : Softmax regression 기본 개념 소개

logistic regression
- 기존의 linear regression 의 형태는 binary classification 이 안됨  
- 0과 1로 결과값이 나올 수 있는 함수를 고안 -> sigmoid or logistic 을 제안
- 그림으로 표현할 경우 아래와 같음
![image.png](img/5-2.png)

![image.png](img/5-3.png)
- 학점 A,B,C를 주는 경우
- A 이거나 아니거나 , B이거나 아니거나, C이거나 아니거나 3개의 각각 다른 선을 그어 구분

![image.png](img/5-4.png)
- 위와 같이 각각 3개의 공식을 만들어 3개의 선을 그을 수 있으나 공식 및 코드가 복잡해짐
- 하나의 matrix 형태로 만들어서 곱을 할 경우, 위와 같이 3 by 1 형태의 vector형태로 결과값이 나옴
- 최종 벡터 값에서 A, B, C 의 값 중 하나를 골라 주는 것이 softmax 함수

#### softmax function
![image.png](img/5-5.png)
- 최종 벡터값(가장 좌측)에서 시그모이드 함수를 적용 - 이를 다시 one-hot-encoding

#### cost function
![image.png](img/5-6.png)
- cross entropy
- L이 실제 정답, S(Y)는 y hat, 두 개사이의 차이가 얼마나 되는지 크로스 엔트로피 함수를 통해 구하게 됨. 

#### lab 06-1 : TensorFlow로 softmax classification 구현하기

In [3]:
import tensorflow as tf

In [4]:
x_data = [[1, 2, 1, 1], [2,1, 3, 2], [3,1,3,4], [4,1,5,5], [1,7,5,5,],[1,2,5,6], [1,6,6,6], [1,7,7,7]]
y_data = [[0,0,1], [0,0,1], [0,0,1], [0,1,0], [0,1,0], [0,1,0], [1,0,0], [1,0,0]]

In [9]:
X  =tf.placeholder("float", [None, 4])
Y = tf.placeholder("float", [None, 3])
nb_classes = 3

In [10]:
W = tf.Variable(tf.random_normal([4, nb_classes]), name = 'weight')
b = tf.Variable(tf.random_normal([nb_classes]), name = 'bias')


In [11]:
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

In [12]:
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis = 1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1).minimize(cost)

In [ ]:
with tf.Session() as sess :
    sess.run(tf.global_variables_initializer())
    
    for step in range(3001) :
        sess.run(optimizer, feed_dict = {X: x_data, Y:y_data})
        if step % 200 == 0 :
            print(step, sess.run(cost, feed_dict = {X: x_data, Y: y_data}))

In [ ]:
a = sess.run(hypothesis, feed_dict = {X : [[1, 11, 7, 9]]})
print(a, sess.run(tf.arg_max(a, 1)))

#### lab 06 -2 : TensorFlow Fancy Softmax classification 구현하기

In [22]:
#logistic classifier 통과 후
logits = tf.matmul(X, W) + b
#softmax function 을 통해 one-hot-encoding 
hypothesis = tf.nn.softmax(logits)

In [23]:
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis = 1))

In [28]:
cost_i = tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = Y_one_hot)
cost = tf.reduce_mean(cost_i)

In [29]:
Y = tf.placeholder(tf.int32, [None, 1]) # 0 ~ 6, shape = (? , 1)
Y_one_hot = tf.one_hot(Y, nb_classes) # one hot shape = (?, 1, 7)
#위의  one_hot 만 했을 경우, 창원의 수가 추가 되어 (?, 1, 7)의 형태가 나옴
#우리가 원하는 형태가 아니므로, tf.reshape로 다시 one_hot 형태를 (?, 7)로 변겅
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes]) #shape = (?, 7)